In [1]:
const N_DIMS = 2 # enter number of dimensions here.
include("higher_dim_gcmc.jl")

NVT (generic function with 1 method)

In [2]:
# For Kr in MIL-91(Al)
site_volume = 12.32 # A^3, motivated by MIL-93(Al)
energy_params = construct_energy_param(ϵ_0=3.0, ϵ=3.0, ϵ_l=3.0)
pressures = vcat(collect(linspace(0.0001, 0.07, 4)), collect(linspace(0.07, 0.1, 15)))
pressures = collect(linspace(0.0, 0.005, 35))
temperature = 77.0 # K
RT = 8.314 * temperature / 1000.0 # kJ/mol for Qst
#site_volume = 372.59 # A^3
n_sites = 10

10

## higher dimension (2D) grand canonical Monte Carlo simulation

In [ ]:
df_gcmc = run_gcmc_isotherm(pressures, energy_params, n_sites, site_volume, temperature, 
        desorption_too=true, samples_per_site=50000);

In [ ]:
using Compose

myplot_n = plot( 
    # ADsorption
    layer(x=df_gcmc[:P][~df_gcmc[:desorption_branch]], y=df_gcmc[:n][~df_gcmc[:desorption_branch]], 
        Geom.point, Geom.line, Theme(default_color=colorant"green")),
    # DEsorption
    layer(x=df_gcmc[:P][df_gcmc[:desorption_branch]], y=df_gcmc[:n][df_gcmc[:desorption_branch]], 
        Geom.point, Geom.line, Theme(default_color=colorant"orange")),
    Guide.xlabel("Pressure, 𝑃 (bar)"),
    Guide.ylabel("⟨𝑛⟩/𝑀"),
    Guide.title("Adsorption isotherm"),
    Theme(background_color=colorant"white", panel_stroke=colorant"black", 
        grid_color=colorant"Gray", line_width=.7mm, 
        major_label_font_size=15pt, minor_label_font_size=14pt,
        minor_label_color=colorant"black", major_label_color=colorant"black", 
        key_title_color=colorant"black", key_label_color=colorant"black"),
     Guide.yticks(ticks=collect(0.0:0.2:1.0))
)

# Linker configurations
myplot_l = plot(
    # GCMC
    # ADsorption
    layer(x=df_gcmc[:P][~df_gcmc[:desorption_branch]], y=df_gcmc[:l][~df_gcmc[:desorption_branch]], 
        Geom.point, Geom.line, Theme(default_color=colorant"green")),
    # DEsorption
    layer(x=df_gcmc[:P][df_gcmc[:desorption_branch]], y=df_gcmc[:l][df_gcmc[:desorption_branch]], 
        Geom.point, Geom.line, Theme(default_color=colorant"orange")),
#     # Inflection pressure
#     xintercept=[inflection_pressure],
#     Geom.vline(),

    Guide.xlabel("Pressure, 𝑃 (bar)"),
    Guide.ylabel("⟨ℓ⟩/(2𝑀)"),
    Guide.title("Linker configurations"),
    Theme(background_color=colorant"white", panel_stroke=colorant"black", 
        grid_color=colorant"Gray", line_width=.7mm, 
        major_label_font_size=15pt, minor_label_font_size=14pt,
        minor_label_color=colorant"black", major_label_color=colorant"black", 
        key_title_color=colorant"black", key_label_color=colorant"black"),
#     Coord.Cartesian(xmax=1.5),
#     Guide.xticks(ticks=collect(0.0:0.5:1.5)),
     Guide.yticks(ticks=collect(0.0:0.2:1.0))
#     Guide.manual_color_key("", 
#         ["GCMC", "Exact"], 
#         [colorant"red", colorant"blue"]
#     )
)

# Isosteric heat of ads
myplot_qst = plot(
    # ADsorption
    layer(x=df_gcmc[:P][~df_gcmc[:desorption_branch]], y=df_gcmc[:qst][~df_gcmc[:desorption_branch]], 
        Geom.point, Geom.line, Theme(default_color=colorant"green")),
    # DEsorption
    layer(x=df_gcmc[:P][df_gcmc[:desorption_branch]], y=df_gcmc[:qst][df_gcmc[:desorption_branch]], 
        Geom.point, Geom.line, Theme(default_color=colorant"orange")),
    Guide.xlabel("Pressure, 𝑃 (bar)"),
#     Guide.xticks(ticks=collect(0.0:0.5:1.5)),
    Guide.ylabel("Q<sub>st</sub> (kJ/mol)"),
#     Coord.Cartesian(xmax=1.5),
    Guide.title("Heat of adsorption"),
    Theme(background_color=colorant"white", panel_stroke=colorant"black", 
        grid_color=colorant"Gray", line_width=.7mm, 
        major_label_font_size=15pt, minor_label_font_size=14pt,
        minor_label_color=colorant"black", major_label_color=colorant"black", 
        key_title_color=colorant"black", key_label_color=colorant"black"),
#     Guide.manual_color_key("", 
#         ["GCMC", "Exact"], 
#         [colorant"red", colorant"blue"]
#     )
)

draw(SVG(30cm, 8cm), hstack(myplot_n, myplot_l, myplot_qst))
# draw(PDF("2D_simulation_low_samples.pdf", 30cm, 8cm), hstack(myplot_n, myplot_l, myplot_qst))

# Single simulation showing jumping back and forth between high and low density state at intermediate pressure.

In [ ]:
p_transition = 0.074434345
p_transition = 0.0744343425
p_transition = 0.074434344
occupancy, linker_states, stats, samples_of_n = gcmc_simulation(p_transition, energy_params, 10, site_volume, 
temperature, samples_per_site=1000000, verboseflag=false, initial_condition="random")

In [ ]:
bimodal_distn = plot(x=samples_of_n, Geom.histogram(bincount=100), Guide.xlabel("∑n<sub>ij</sub>/𝑀"), Guide.ylabel("# MC samples"),
    Coord.Cartesian(xmax=100.0),
    Theme(background_color=colorant"white", panel_stroke=colorant"black", 
        grid_color=colorant"Gray", line_width=.7mm, 
        major_label_font_size=15pt, minor_label_font_size=14pt,
        minor_label_color=colorant"black", major_label_color=colorant"black", 
        key_title_color=colorant"black", key_label_color=colorant"black")
)
draw(SVG(10cm, 8cm), bimodal_distn)
draw(PDF("bimodal_n_samples.pdf", 10cm, 8cm), bimodal_distn)

# Tests

In [3]:
n_sites = 3
n_dims = 2
linker_states = [zeros(Int, [n_sites for i = 1:n_dims]...) for k = 1:n_dims]
occupancy = zeros(Int, [n_sites for k = 1:n_dims]...)
energy_params = construct_energy_param(ϵ_0=10.0, ϵ=2.0, ϵ_l=3.0)
occupancy[1, 2] = 1
@test_approx_eq adsorbate_energy([1, 2], linker_states, occupancy, energy_params) -energy_params.ϵ_0
linker_states[1][1, 2] = 1
@test_approx_eq adsorbate_energy([1, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0-energy_params.ϵ)
linker_states[1][2, 2] = 1
@test_approx_eq adsorbate_energy([1, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0-2*energy_params.ϵ)
linker_states[2][1, 2] = 1
@test_approx_eq adsorbate_energy([1, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0-3*energy_params.ϵ)
linker_states[2][1, 3] = 1
@test_approx_eq adsorbate_energy([1, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0-4*energy_params.ϵ)

In [4]:
# now one on boundary, start over.
linker_states = [zeros(Int, [n_sites for i = 1:n_dims]...) for k = 1:n_dims]
occupancy = zeros(Int, [n_sites for k = 1:n_dims]...)
occupancy[3, 2] = 1
@test_approx_eq adsorbate_energy([3, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0)
linker_states[1][3, 2] = 1
@test_approx_eq adsorbate_energy([3, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0 - energy_params.ϵ)
linker_states[1][1, 2] = 1
@test_approx_eq adsorbate_energy([3, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0 - 2*energy_params.ϵ)
linker_states[2][3, 2] = 1
@test_approx_eq adsorbate_energy([3, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0 - 3*energy_params.ϵ)
linker_states[2][3, 3] = 1
@test_approx_eq adsorbate_energy([3, 2], linker_states, occupancy, energy_params) (-energy_params.ϵ_0 - 4*energy_params.ϵ)


In [5]:
linker_states = [zeros(Int, [n_sites for i = 1:n_dims]...) for k = 1:n_dims]
occupancy = zeros(Int, [n_sites for k = 1:n_dims]...)
occupancy[3, 3] = 1
linker_states[1][3, 3] = 1
linker_states[2][3, 3] = 1
linker_states[2][3, 1] = 1
linker_states[1][1, 3] = 1
@test_approx_eq adsorbate_energy([3, 3], linker_states, occupancy, energy_params) (-energy_params.ϵ_0 - 4*energy_params.ϵ)

In [6]:
linker_states = [zeros(Int, [n_sites for i = 1:n_dims]...) for k = 1:n_dims]
occupancy = zeros(Int, [n_sites for k = 1:n_dims]...)
@test_approx_eq  linker_energy(1, [2, 2], linker_states, occupancy, energy_params) 0.0
linker_states[1][2, 2] = 1
@test_approx_eq  linker_energy(1, [2, 2], linker_states, occupancy, energy_params) energy_params.ϵ_l
occupancy[2, 2] = 1
@test_approx_eq  linker_energy(1, [2, 2], linker_states, occupancy, energy_params) (energy_params.ϵ_l - energy_params.ϵ)
linker_states[2][2, 2] = 1
@test_approx_eq  linker_energy(1, [2, 2], linker_states, occupancy, energy_params) (energy_params.ϵ_l - energy_params.ϵ)
occupancy[2, 1] = 1
@test_approx_eq  linker_energy(2, [2, 2], linker_states, occupancy, energy_params) (energy_params.ϵ_l - 2*energy_params.ϵ)
@test_approx_eq  linker_energy(1, [2, 2], linker_states, occupancy, energy_params) (energy_params.ϵ_l - energy_params.ϵ)
occupancy[1, 2] = 1
@test_approx_eq  linker_energy(1, [2, 2], linker_states, occupancy, energy_params) (energy_params.ϵ_l - 2*energy_params.ϵ)
occupancy[1, n_sites] = 1
linker_states[2][1, 1] = 1
@test_approx_eq  linker_energy(2, [1, 1], linker_states, occupancy, energy_params) (energy_params.ϵ_l - energy_params.ϵ)
